In [1]:
import numpy as np

In [239]:
class GameState (object):
    def __init__ (self, board=np.zeros(9, dtype=np.int8), player=1):
        self.board =  board
        self.player = player
        
    def new_game(self):
        self.board = np.zeros(9, dtype=np.int8)
        self.player = 1

    def get_legal_actions(self):
        actions = []
        if not self.is_game_over():
            actions, = np.where(self.board==0)
        return actions

    def is_game_over(self):
        # check if board is full
        if 0 not in self.board:
            return True
        # check if anyone won
        elif self.game_result() != 0:
            return True
        return False
    
#     def game_result(self):
#         board = self.board.reshape((3,3))
#         # check rows
#         for i in range(3):
#             value = int(np.sum(board[i,:])/3)
#             if np.absolute(value) == 1:
#                 return value         
#         # check columns
#         for i in range(3):
#             value = int(np.sum(board[:,i])/3)
#             if np.absolute(value) == 1:
#                 return value    
#         # check diagonal 1
#         value = int((board[0][0] + board[1][1] + board[2][2])/3)
#         if np.absolute(value) == 1:
#             return value
#         # check diagonal 2
#         value = int((board[0][2] + board[1][1] + board[2][0])/3)
#         if np.absolute(value) == 1:
#             return value
#         # otherwise it's a tie
#         return 0
    
    def game_result(self):
        winningLines = np.array([[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]])
        board = self.board
        for line in winningLines:
            if len(set(board[line])) == 1:
                return board[line][0]
        return 0
    
    
    def move(self, action):
        board = np.copy(self.board)
        board[action] = self.player
        player = -1*self.player
        return GameState(board, player)
            
    def select_random_action(self):
        return np.random.choice(self.get_legal_actions())

    def get_board(self):
        return self.board

    def get_player(self):
        return self.player

    def __eq__(self, other):
        return ((np.array_equal(self.board, other.board)) and (self.player == other.player))

    def __ne__(self, other):
        return not self.__eq__(other)
    
def print_board(board):
    board = board.reshape((3,3))
    symbol = {1: 'x', -1: 'o', 0: ' '}
    for row in board:
        print('|'+symbol[row[0]]+'|'+symbol[row[1]]+'|'+symbol[row[2]]+'|')
        
    

In [3]:
# from Game import*
# state = GameState(board=np.zeros(9, dtype=np.int8), player=1)
state = GameState()
state.new_game()
print("GAME\n")
print(state.get_board().reshape((3,3)))
while not state.is_game_over():
    action = state.select_random_action()
    print("\nPlayer to play: %d" % (state.get_player()))
    print("Chosen move:    %d\n" % (action))
    state.move(action)
    print(state.get_board().reshape((3,3)))
print("\nGame result: %d" % (state.game_result()))

GAME

[[0 0 0]
 [0 0 0]
 [0 0 0]]

Player to play: 1
Chosen move:    3

[[0 0 0]
 [1 0 0]
 [0 0 0]]

Player to play: -1
Chosen move:    0

[[-1  0  0]
 [ 1  0  0]
 [ 0  0  0]]

Player to play: 1
Chosen move:    4

[[-1  0  0]
 [ 1  1  0]
 [ 0  0  0]]

Player to play: -1
Chosen move:    1

[[-1 -1  0]
 [ 1  1  0]
 [ 0  0  0]]

Player to play: 1
Chosen move:    5

[[-1 -1  0]
 [ 1  1  1]
 [ 0  0  0]]

Game result: 1


In [4]:
from collections import defaultdict
import copy
# from Game import *

In [180]:
class Node (object):
    def __init__ (self, state, parent=None, parent_action=None):
        self.state = state
        self.parent = parent
        self.parent_action = parent_action
        self.children = []
        self.number_of_visits = 0
        self.results = defaultdict(int)
        self.results[1] = 0
        self.results[-1] = 0
        self.results[0] = 0
        self.untried_actions = self.state.get_legal_actions()

    def n(self):
        return self.number_of_visits

    def reward(self):
        reward = (self.results[1]-self.results[-1])/self.n()
        return reward*self.parent.state.get_player()

    def is_terminal_node(self):
        return self.state.is_game_over()
    
    def is_fully_expanded(self):
        return len(self.untried_actions) == 0

    def expand(self):
        action = self.untried_actions[0]
        self.untried_actions = self.untried_actions[1:]
        new_state = self.state.move(action)
        child = Node(new_state, parent=self, parent_action=action)
        self.children.append(child)
        return child

    def best_child(self, c_param=1.0):
        UCT = [(c.reward() + c_param*np.sqrt(2*np.log(self.n())/c.n())) for c in self.children]
        return self.children[np.argmax(UCT)]
    
    def most_visited_child(self):
        visits = [c.n() for c in self.children]
        return self.children[np.argmax(visits)]
    
    def backpropagate(self, result):
        self.number_of_visits += 1.
        self.results[result] += 1.
        if self.parent:
            self.parent.backpropagate(result)
        
    
    def tree_policy(self, c_param=1.0):
        current_node = self
        while not current_node.is_terminal_node():
            if not current_node.is_fully_expanded():
                return current_node.expand()
            else:
                current_node = current_node.best_child(c_param)
        return current_node
    
    def rollout(self):
        current_state = copy.deepcopy(self.state)
        while not current_state.is_game_over():
            action = current_state.select_random_action()
            current_state = current_state.move(action)
        return current_state.game_result()
    
class MCTS (object):
    def __init__ (self, state, c_param=1.):
        self.root = Node(state)
        self.c_param = c_param
        
    def sweep(self):
        v = self.root.tree_policy(self.c_param)
        reward = v.rollout()
        v.backpropagate(reward)
        
    def best_move(self):
        return self.root.best_child(c_param=0.)
    
    def play_move(self, state):
        idx = self.root.children.index(state)
        self.root = self.root.children[idx]
        self.root.parent = None
        
    def root_state(self):
        return self.root.state
    

In [246]:
# np.random.seed(1234)
state = GameState()
state.new_game()
mcts = MCTS(state, c_param=1.)
print("STARTING BOARD:\n")
print_board(state.get_board())

turn = 1
while not state.is_game_over():
    print('\nMOVE %d\n' % (turn))
    
    for n in range(100):
        mcts.sweep()
        
    best_move = mcts.best_move()
    mcts.play_move(best_move)
    state = mcts.root_state()
    
    print_board(state.get_board())
    
    turn += 1

result = state.game_result()
if result==1:
    print('\nPlayer 1 won!')
elif result==-1:
    print('\nPlayer 2 won!')
else:
    print('\nThe game ended in a tie!')

STARTING BOARD:

| | | |
| | | |
| | | |

MOVE 1

| | | |
| |x| |
| | | |

MOVE 2

| | | |
| |x| |
|o| | |

MOVE 3

| | | |
| |x| |
|o|x| |

MOVE 4

| |o| |
| |x| |
|o|x| |

MOVE 5

|x|o| |
| |x| |
|o|x| |

MOVE 6

|x|o| |
| |x| |
|o|x|o|

MOVE 7

|x|o| |
|x|x| |
|o|x|o|

MOVE 8

|x|o| |
|x|x|o|
|o|x|o|

MOVE 9

|x|o|x|
|x|x|o|
|o|x|o|

The game ended in a tie!


In [ ]:
# np.random.seed(1234)
state = GameState()
state.new_game()
mcts = MCTS(state, c_param=1.)
print("STARTING BOARD:\n")
print_board(state.get_board())

turn = 1
while not state.is_game_over():
    player = state.get_player()
    print('\nMOVE %d\n' % (turn))
    
    for n in range(100):
        mcts.sweep()
        
    if player==1:
        legal_actions = state.get_legal_actions()
        print('Legal moves: ', legal_actions)
        move = int(input ("Make your move: "))
        while int(move) not in 
        state = move()
    best_move = mcts.best_move()
    mcts.play_move(best_move)
    state = mcts.root_state()
    
    print_board(state.get_board())
    
    turn += 1

result = state.game_result()
if result==1:
    print('\nPlayer 1 won!')
elif result==-1:
    print('\nPlayer 2 won!')
else:
    print('\nThe game ended in a tie!')

In [6]:
state = GameState()
state.new_game()
state = GameState(board=np.array([1,1,-1,-1,0,0,0,0,0]))


In [108]:
state = GameState()
state.new_game()
print("GAME\n")
print(state.get_board().reshape((3,3)))
np.random.seed(1234)
tree = Node(state)
print(tree.rollout())
print(tree.state.get_board().reshape((3,3)))

GAME

[[0 0 0]
 [0 0 0]
 [0 0 0]]
0
[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [178]:
board = np.array([-1,0,-1,1,1,0,0,0,0])
state = GameState(board=board, player=1)
print(state.get_board().reshape((3,3)))

mcts = MCTS(state, c_param=1.)
print(mcts.root.state.get_legal_actions())
print(mcts.root.untried_actions)
for n in range(100):
    mcts.sweep()
#     print(mcts.root.untried_actions)
#     print(mcts.root.state.get_board().reshape((3,3)))
best_move = mcts.best_move()
print(best_move.state.get_board().reshape((3,3)))
print(best_move.parent_action)

mcts.play_move(best_move)
print(mcts.root.state.get_board().reshape((3,3)))

[[-1  0 -1]
 [ 1  1  0]
 [ 0  0  0]]
[1 5 6 7 8]
[1 5 6 7 8]
[[-1  0 -1]
 [ 1  1  1]
 [ 0  0  0]]
5
[[-1  0 -1]
 [ 1  1  1]
 [ 0  0  0]]


In [139]:
board = np.array([-1,0,-1,1,1,0,0,0,0])
state = GameState(board=board, player=1)
print(state.get_board().reshape((3,3)))
mcts = MCTS(state, c_param=1.)
print(mcts.root.state.get_legal_actions())
print(mcts.root.untried_actions)

for n in range(9):
    child = mcts.root.expand()
    print(mcts.root.untried_actions)
    print(mcts.root.state.get_board().reshape((3,3)))
    print(child.state.get_board().reshape((3,3)))
    print(mcts.root.children[n].state.get_board().reshape((3,3)))
    
#     print(mcts.root.parent.state.get_board().reshape((3,3)))

[[-1  0 -1]
 [ 1  1  0]
 [ 0  0  0]]
[1 5 6 7 8]
[1 5 6 7 8]
[5 6 7 8]
[[-1  0 -1]
 [ 1  1  0]
 [ 0  0  0]]
[[-1  1 -1]
 [ 1  1  0]
 [ 0  0  0]]
[[-1  1 -1]
 [ 1  1  0]
 [ 0  0  0]]
[6 7 8]
[[-1  0 -1]
 [ 1  1  0]
 [ 0  0  0]]
[[-1  0 -1]
 [ 1  1  1]
 [ 0  0  0]]
[[-1  0 -1]
 [ 1  1  1]
 [ 0  0  0]]
[7 8]
[[-1  0 -1]
 [ 1  1  0]
 [ 0  0  0]]
[[-1  0 -1]
 [ 1  1  0]
 [ 1  0  0]]
[[-1  0 -1]
 [ 1  1  0]
 [ 1  0  0]]
[8]
[[-1  0 -1]
 [ 1  1  0]
 [ 0  0  0]]
[[-1  0 -1]
 [ 1  1  0]
 [ 0  1  0]]
[[-1  0 -1]
 [ 1  1  0]
 [ 0  1  0]]
[]
[[-1  0 -1]
 [ 1  1  0]
 [ 0  0  0]]
[[-1  0 -1]
 [ 1  1  0]
 [ 0  0  1]]
[[-1  0 -1]
 [ 1  1  0]
 [ 0  0  1]]


IndexError: index 0 is out of bounds for axis 0 with size 0